In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch, torchvision
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights
from torchvision import datasets
import matplotlib.pyplot as plt
from torcheval.metrics import BinaryAccuracy

torch.set_default_device('cpu')
device = torch.device('cpu')
torch.manual_seed(0)
batch_size = 64

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((256, 256)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [ ]:
train = datasets.OxfordIIITPet("", split="trainval", transform=transform, target_types="binary-category", download=True)
test = datasets.OxfordIIITPet("", split="test", transform=transform, target_types="binary-category", download=True)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*61*61, 500)
        self.fc2 = nn.Linear(500, 120)
        self.fc3 = nn.Linear(120, 84)
        self.fc4 = nn.Linear(84, 1)

    def forward(self, x):
        x = x.to(device)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        x = torch.nn.BatchNorm1d(x.shape[1])(x)
        x = F.sigmoid(self.fc4(x))
        x = torch.flatten(x, 0)
        return x
net = Net().to(device)

In [ ]:
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
metric = BinaryAccuracy()

In [ ]:
trainloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=5)

In [ ]:
batches = train.__len__() // batch_size
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device).to(torch.float32)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        metric.update(outputs, labels)
        if i % 10 == 9:
            print('[%d, %5d/%d] loss: %.3f, acc:%.3f' %
                (epoch + 1, i + 1, batches, running_loss / 10, metric.compute()))
            running_loss = 0.0
        elif i == batches - 1:
            print('[%d, %5d/%d] loss: %.3f, acc:%.3f' %
                (epoch + 1, i + 1, batches, running_loss / (batches % 10), metric.compute()))
            running_loss = 0.0

print('Finished Training')

In [ ]:
testloader = torch.utils.data.DataLoader(test, batch_size=10, num_workers=2)
metric.reset()
for i, data in enumerate(testloader, 0):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = net(inputs)
    metric.update(outputs, labels)
print(f"Test accuracy: {metric.compute()}")